In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline

In [2]:
import dipy.reconst.dti as dti
import dipy.reconst.dki as dki
import dipy.reconst.cross_validation as xval
import dipy.core.gradients as dpg
from dipy.segment.mask import median_otsu

In [3]:
import nibabel as nib

In [4]:
import tools
import imp
imp.reload(tools)
from tools import resample_volume

In [5]:
dwi_img = nib.load('/home/ubuntu/data/991267/sess/dwi/dwi.nii.gz')
gtab = dpg.gradient_table('/home/ubuntu/data/991267/sess/dwi/dwi.bvals', 
                          '/home/ubuntu/data/991267/sess/dwi/dwi.bvecs',
                          b0_threshold=10)

In [6]:
data = dwi_img.get_data()

In [7]:
t1_img = nib.load('/home/ubuntu/data/991267/sess/anat/T1w_acpc_dc.nii.gz')
label_img = nib.load('/home/ubuntu/data/991267/sess/anat/aparc+aseg.nii.gz')

In [8]:
resamp_t1 = resample_volume(t1_img, dwi_img)
resamp_label = resample_volume(label_img, dwi_img)

In [9]:
resamp_label.shape, resamp_t1.shape, dwi_img.shape

((145, 174, 145), (145, 174, 145), (145, 174, 145, 288))

In [10]:
t1_data = resamp_t1.get_data()
label_data = resamp_label.get_data()

In [11]:
# Cerebral white matter in both hemispheres + corpus callosum
wm_mask = (label_data==41) | (label_data==2) | (label_data==86)

In [12]:
dki_model = dki.DiffusionKurtosisModel(gtab)
dti_model = dti.TensorModel(gtab)

In [13]:
def calc_cod(model, data, mask, folds=2):
    pred = xval.kfold_xval(model, data, folds, mask=mask)
    cod = xval.coeff_of_determination(pred, data)
    return cod

In [ ]:
cod_dki = calc_cod(dki_model, data, wm_mask) 
cod_dti = calc_cod(dti_model, data, wm_mask)

In [ ]:
plt.hist(cod_dti[np.isfinite(cod_dki)] - cod_dki[np.isfinite(cod_dki)], bins=1000)
plt.xlim([-15, 15])